In [1]:
import requests
import json
from bs4 import BeautifulSoup
import time
import re
import pandas as pd
import numpy as np
from pymongo import MongoClient

In [2]:
url = "http://scholarworks.unist.ac.kr/browse-researcher"
data = {
    "sortType": "English",
    "page": 1,
    "offset": 0
}

In [3]:
## 데이터 정보를 보내지 않았다!
r = requests.get(url, params=data)
text = r.text
soup = BeautifulSoup(text, 'lxml')

In [4]:
li = soup.findAll("li")[16:76]
li[0:5]

[<li>
 <a href="researcher-profile?ep=2741">Ahn, Sangjoon
 
 						</a>
 </li>, <li>
 <a href="researcher-profile?ep=2781">Amblard, Francois
 
 						</a>
 </li>, <li>
 <a href="researcher-profile?ep=2641">An, Kwangjin
 
 						</a>
 </li>, <li>
 <a href="researcher-profile?ep=1881">Au, Tsz-Chiu
 
 						</a>
 </li>, <li>
 <a href="researcher-profile?ep=2041">Bae, Hantaek
 
 						</a>
 </li>]

In [5]:
link = []
name = []

def makelink(pg):
    url = "http://scholarworks.unist.ac.kr/browse-researcher"
    data = {
        "sortType": "English",
        "page": pg,
        "offset": 0
    }
    r = requests.get(url, params=data)
    text = r.text
    soup = BeautifulSoup(text, 'lxml')
    for i in soup.findAll("li")[16:76]:
        link.append(i.find("a")['href'])
        name.append(i.getText().replace("\n","").replace("\r","").replace("\t",""))

In [6]:
for i in range(5):
    makelink(i+1)

In [7]:
def makelink6():
    url = "http://scholarworks.unist.ac.kr/browse-researcher"
    data = {
        "sortType": "English",
        "page": 6,
        "offset": 0
    }
    r = requests.get(url, params=data)
    text = r.text
    soup = BeautifulSoup(text, 'lxml')
    for i in soup.findAll("li")[16:28]:
        link.append(i.find("a")['href'])
        name.append(i.getText().replace("\n","").replace("\r","").replace("\t",""))

In [8]:
makelink6()

In [9]:
link[311], name[311]

('researcher-profile?ep=1299', 'Zhang, Lu')

In [10]:
len(link)

312

In [11]:
client = MongoClient()
db = client.aboutdepart

In [12]:
db.scholar.drop()

In [13]:
for i, k in zip(link,name):    
    document = {}
    document["link"] = i
    document["name"] = k
    db.scholar.insert_one(document)

In [14]:
db.scholar.count()

/Users/seungheondoh/anaconda3/envs/python3.6/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: count is deprecated. Use estimated_document_count or count_documents instead. Please note that $where must be replaced by $expr, $near must be replaced by $geoWithin with $center, and $nearSphere must be replaced by $geoWithin with $centerSphere
  """Entry point for launching an IPython kernel.


312

In [15]:
Contents = db.scholar.find()
Contents[5]

{'_id': ObjectId('5bf8068320c66c09bf8e070e'),
 'link': 'researcher-profile?ep=1249',
 'name': 'Bae, Joonbum'}

-------------------------------------

### db.scholar에서 저장한 link를 타고 url을 호출합니다.

In [16]:
url = "http://scholarworks.unist.ac.kr/"+str(link[1])
r = requests.get(url)
soup = BeautifulSoup(r.text, 'lxml')

In [30]:
soup.find("img",{"alt":"사진"})['src']

'http://114.70.1.191/rims/upload/image/201508/user_profile_5929480789290965124100358.jpg'

In [18]:
soup.find('div',{'class' : "research_info_view"})

<div class="research_info_view">
<div class="researcher_idbox">
<ul>
<li>
<a href="http://www.researcherid.com/rid/F-4552-2015" id="F-4552-2015" target="_blank"><img alt="ResearcherID" class="tooltip" src="/image/common/research_icon02.gif" title="ResearcherID"/></a>
</li>
<li>
<a href="https://www.researchgate.net/profile/Francois_Amblard" id="https://www.researchgate.net/profile/Francois_Amblard" target="_blank"><img alt="ResearchGate" class="tooltip" src="/image/common/research_icon03.gif" title="ResearchGate"/></a>
</li>
<li>
<a href="http://orcid.org/0000-0002-7247-3873" id="http://orcid.org/0000-0002-7247-3873" target="_blank"><img alt="ORCiD" class="tooltip" src="/image/common/research_icon01.gif" title="ORCiD"/></a>
</li>
<li>
<a href="http://www.scopus.com/authid/detail.url?authorId=7004730960" id="http://www.scopus.com/authid/detail.url?authorId=7004730960" target="_blank"><img alt="Scopus" class="tooltip" src="/image/common/SCPID_icon.png" title="Scopus" width="34"/></a>
</l

In [22]:
soup.find('p',{'class':'neme_p'}).getText()

'\r\n\t\t\tAmblard, Francois\xa0(AmblardFrancois)\r\n\t\t\t\r\n\t\t'

In [23]:
soup.find('div',{'class' : "research_info_view"}).findAll('dl')

[<dl>
 <dt>Department</dt>
 <dd>
 			School of Life Sciences(생명과학부)
 			</dd>
 </dl>, <dl>
 <dt>Research Interests</dt>
 <dd>
 			Biology, Cancer Biology
        		</dd>
 </dl>, <dl>
 <dt>Lab</dt>
 <dd>School of Life Science</dd>
 </dl>, <dl>
 <dt>Website</dt>
 <dd><a href="#" target="_blank"></a></dd>
 </dl>]

In [24]:
#name
soup.find('p',{'class':'neme_p'}).getText().replace("\xa0","").replace("\r","").replace("\t","").replace("\n","")

'Amblard, Francois(AmblardFrancois)'

In [25]:
#department
soup.find('div',{'class' : "research_info_view"}).findAll('dl')[0].find('dd').getText().replace("\r","").replace("\t","").replace("\n","")

'School of Life Sciences(생명과학부)'

In [26]:
#Research Interest
soup.find('div',{'class' : "research_info_view"}).findAll('dl')[1].find('dd').getText().replace("\r","").replace("\t","").replace("\n","").strip()

'Biology, Cancer Biology'

In [27]:
#Lab
soup.find('div',{'class' : "research_info_view"}).findAll('dl')[2].find('dd').getText()

'School of Life Science'

In [28]:
#Website
soup.find('div',{'class' : "research_info_view"}).findAll('dl')[3].find('dd').getText()

''

### 학술지 정보

In [29]:
soup.find('div',{'class' : "researcher_idbox"})

<div class="researcher_idbox">
<ul>
<li>
<a href="http://www.researcherid.com/rid/F-4552-2015" id="F-4552-2015" target="_blank"><img alt="ResearcherID" class="tooltip" src="/image/common/research_icon02.gif" title="ResearcherID"/></a>
</li>
<li>
<a href="https://www.researchgate.net/profile/Francois_Amblard" id="https://www.researchgate.net/profile/Francois_Amblard" target="_blank"><img alt="ResearchGate" class="tooltip" src="/image/common/research_icon03.gif" title="ResearchGate"/></a>
</li>
<li>
<a href="http://orcid.org/0000-0002-7247-3873" id="http://orcid.org/0000-0002-7247-3873" target="_blank"><img alt="ORCiD" class="tooltip" src="/image/common/research_icon01.gif" title="ORCiD"/></a>
</li>
<li>
<a href="http://www.scopus.com/authid/detail.url?authorId=7004730960" id="http://www.scopus.com/authid/detail.url?authorId=7004730960" target="_blank"><img alt="Scopus" class="tooltip" src="/image/common/SCPID_icon.png" title="Scopus" width="34"/></a>
</li>
</ul>
</div>

In [30]:
for i in soup.find('div',{'class' : "researcher_idbox"}).findAll('a'):
    if "http://www.scopus.com/authid/detail.url?" in i['href']:
        print(i['href'].replace("http://www.scopus.com/authid/detail.url?authorId=",""))

7004730960


## Final
db 스키마

- id : 스코퍼스 고유 id
- department : 소속학과
- ResearchInterest : 관심단어
- Lab : 연구실 명
- Website : 웹사이트 페이지

In [56]:
db.scholarworksfront.drop()

In [57]:
link[1]

'researcher-profile?ep=2781'

In [58]:
def savedata(a): 
    s = a.strip()
    url = "http://scholarworks.unist.ac.kr/"+s
    r = requests.get(url)
    soup = BeautifulSoup(r.text, 'lxml')
    dl = soup.find('div',{'class' : "research_info_view"}).findAll('dl')
    document ={}
    try:
        for i in soup.find('div',{'class' : "researcher_idbox"}).findAll('a'):
            if "http://www.scopus.com/authid/detail.url?" in i['href']:
                document['id']= i['href'].replace("http://www.scopus.com/authid/detail.url?authorId=","")
    except:
        document['id'] = ""
    document['Name'] = soup.find('p',{'class':'neme_p'}).getText().replace("\xa0","").replace("\r","").replace("\t","").replace("\n","")
    document['department']= dl[0].find('dd').getText().replace("\r","").replace("\t","").replace("\n","")
    document['ResearchInterest']=dl[1].find('dd').getText().replace("\r","").replace("\t","").replace("\n","").strip()
    document['Lab']=dl[2].find('dd').getText()
    document['Website']=dl[3].find('dd').getText()
    document['imglink']=soup.find("img",{"alt":"사진"})['src']
    db.scholarworksfront.insert_one(document)

In [59]:
for i in link:
    savedata(i)

In [60]:
db.scholarworksfront.count()

/Users/seungheondoh/anaconda3/envs/python3.6/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: count is deprecated. Use estimated_document_count or count_documents instead. Please note that $where must be replaced by $expr, $near must be replaced by $geoWithin with $center, and $nearSphere must be replaced by $geoWithin with $centerSphere
  """Entry point for launching an IPython kernel.


312